In [51]:
import warnings
from crimm import fetch_rcsb
from crimm.StructEntities.OrganizedModel import OrganizedModel
from crimm.Modeller.Solvator import Solvator

from crimm.Fetchers import fetch_rcsb
from crimm.Modeller import TopologyGenerator
from crimm.Modeller.CoordManipulator import CoordManipulator
from crimm.Modeller.LoopBuilder import ChainLoopBuilder
from crimm.Adaptors.PropKaAdaptors import PropKaProtonator
from crimm.Utils.StructureUtils import get_coords

import pycharmm
from pycharmm.settings import set_verbosity as pcm_set_verbosity
from pycharmm import write as pcm_write
from pycharmm import NonBondedScript

from crimm.Adaptors.pyCHARMMAdaptors import (
    load_chain, load_topology, load_water, load_ions, load_ligands,
    create_water_hs_from_charmm, fetch_coords_from_charmm, patch_disu_from_model,
    sd_minimize
)

import pycharmm.minimize as minimize
import pycharmm.energy as energy
from pycharmm import coor, crystal, image, cons_harm, cons_fix, generate

In [23]:
# cgenff excutable path is used later in topology generation
CGENFF_PATH = "/export/app/cgenff/silcsbio.2024.1/cgenff/cgenff"
PDBID = '4E0J' #'5iev'#'1bg8' #'3q4k' #'4pti' #'2HZI' 

## Fetch from RCSB

The fetch_rcsb has be updated that it takes argument `organize`. When it is `True`, the structure will be organized into chain types, and an `OrganizedModel` will be returned instead of the unorganized structure entity.

In [33]:
structure = fetch_rcsb(
    '4pti',
    include_solvent=True, # We want to incude crystallographic water
    use_bio_assembly=True,
    organize=False,
    first_model_only=False
)

In [35]:
## the OrganinzedModel is improved with more feature and APIs
## and has become the main object that deals with modeling and interfacing pyCHARMM
## There will be another notebook showcasing more about OrganizedModel

model = OrganizedModel(structure)
model

NGLWidget()

<OrganizedModel model=4PTI Polypeptide(L)=1 Solvent=1 >
	│
	├───<Polypeptide(L) id=A Residues=58>
	├──────Description: TRYPSIN INHIBITOR
	│
	├───<Solvent id=B Residues=60>
	├──────Residue ID(s): HOH
	├──────Description: water


In [36]:
## Place the model center to (0, 0, 0) and place the principle axis along x-axis
coord_man = CoordManipulator()
coord_man.load_entity(model)
coord_man.orient_coords()

In [37]:
# build missing loops if exist
for chain in model.protein:
    if not chain.is_continuous():
        # chain can be built in place now by specifying `inplace = True`
        looper = ChainLoopBuilder(chain, inplace = True)
        # looper.build_from_homology(max_num_match=10, identity_score_cutoff=0.95)
        # missing terminals will also be built if `include_terminal = True`
        looper.build_from_alphafold(include_terminal = False)

In [38]:
chain.is_continuous()

True

## Generate Topology

Topology generation is simplified by using organized model. If `cgenff_path` is specified, ligands are also generated

In [9]:
TopologyGenerator?

Init signature: TopologyGenerator(cgenff_excutable_path=None, cgenff_output_path=None)
Docstring:     
Class for generating topology elements from the topology definition
and parameters.
The topology definition the parameters are loaded from the CHARMM 36 RTF and 
PRM files.
If the cgenff_excutable_path is provided, the topology definition and 
parameters for the heterogen residues are generated by cgenff, and ligand mol2
file and cgenff rtf file will be saved if cgenff_output_path is specified.
Init docstring: Initialize the topology generator.
File:           ~/crimm/crimm/Modeller/TopoLoader.py
Type:           type
Subclasses:     

In [39]:
topo = TopologyGenerator(
    cgenff_excutable_path=CGENFF_PATH,
    cgenff_output_path='./cgenff/'
)
topo.generate_model(
    model,
    prot_first_patch='ACE',
    prot_last_patch='CT3',
    coerce=True
)

/home/truman/crimm/crimm/Modeller/TopoLoader.py:1188: UserWarning: Atom OXT from (58, 'ALA') is not defined in the topology file!
  warnings.warn(
/home/truman/crimm/crimm/Modeller/TopoLoader.py:743: UserWarning: No cmap found in <Polypeptide(L) id=A Residues=58>.
  warnings.warn(
/home/truman/crimm/crimm/Modeller/TopoLoader.py:470: UserWarning: Disulfide bond found between <Residue CYS het=  resseq=5 icode= > and <Residue CYS het=  resseq=55 icode= >! Removing hydrogen HG1 from the residues.
  warnings.warn(
/home/truman/crimm/crimm/Modeller/TopoLoader.py:470: UserWarning: Disulfide bond found between <Residue CYS het=  resseq=14 icode= > and <Residue CYS het=  resseq=38 icode= >! Removing hydrogen HG1 from the residues.
  warnings.warn(
/home/truman/crimm/crimm/Modeller/TopoLoader.py:470: UserWarning: Disulfide bond found between <Residue CYS het=  resseq=30 icode= > and <Residue CYS het=  resseq=51 icode= >! Removing hydrogen HG1 from the residues.
  warnings.warn(


### Printing out the TOPPAR and their Versions Being Used and Loaded

In [40]:
for rtf_type, topo_loader in topo.res_def_dict.items():
    print(rtf_type, 'toppar version:', topo_loader.rtf_version)

cgenff toppar version: 36.1
protein toppar version: 36.2
water_ions toppar version: 31.1


In [41]:
TopologyGenerator?

Init signature: TopologyGenerator(cgenff_excutable_path=None, cgenff_output_path=None)
Docstring:     
Class for generating topology elements from the topology definition
and parameters.
The topology definition the parameters are loaded from the CHARMM 36 RTF and 
PRM files.
If the cgenff_excutable_path is provided, the topology definition and 
parameters for the heterogen residues are generated by cgenff, and ligand mol2
file and cgenff rtf file will be saved if cgenff_output_path is specified.
Init docstring: Initialize the topology generator.
File:           ~/crimm/crimm/Modeller/TopoLoader.py
Type:           type
Subclasses:     

In [42]:
# Organized model
model

NGLWidget()

<OrganizedModel model=4PTI Polypeptide(L)=1 Solvent=1 >
	│
	├───<Polypeptide(L) id=A Residues=58>
	├──────Description: TRYPSIN INHIBITOR
	│
	├───<Solvent id=B Residues=60>
	├──────Residue ID(s): HOH
	├──────Description: water


Modified residue creates breaks in chain after coersion

In [43]:
# Protonation 
protonator = PropKaProtonator(topo, pH = 6)
protonator.load_model(model)
# if there is any pathching applied in crimm, CHARMM PATCH command will be automatically run 
# when protein chains are loaded into CHARMM
protonator.apply_patches()

Unexpected number (14) of atoms in residue ARG   1 A   in conformation 1A
Unexpected number (7) of atoms in residue ALA  58 A   in conformation 1A


No protonation patches to apply on chain A.


In [44]:
## All the topology definition and parameter generated for the model is 
## organized in model.topology_loader. load_topology() takes care of 
## loading sequence and only loads what is need for the model
load_topology(model.topology_loader)

  
 CHARMM>     read rtf card -
 CHARMM>     name /tmp/tmpk4emk5s0
 VOPEN> Attempting to open::/tmp/tmpk4emk5s0::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> * PROTEIN RTF LOADED FROM CRIMM
 TITLE> 36  2
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param card -
 CHARMM>     name /tmp/tmptb4707bx -
 CHARMM>     flex
 VOPEN> Attempting to open::/tmp/tmptb4707bx::

          PARAMETER FILE BEING READ FROM UNIT 91
 TITLE> * PROTEIN PRM LOADED FROM CRIMM
 TITLE> *>>>> CHARMM36 ALL-HYDROGEN PARAMETER FILE FOR PROTEINS <<<<<<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> JAN. 2016 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 PARMIO> NONBOND, HBOND lists and IMAGE atoms cleared.
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM> 

In [45]:
for chain in model.protein:
    load_chain(chain)
# we need to patch disulfide bonds in CHARMM
# the disu info is stored in model under model.connect_dict
patch_disu_from_model(model)

  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmpniu0rk4e
 VOPEN> Attempting to open::/tmp/tmpniu0rk4e::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --    58 RESIDUES
          ARG PRO ASP PHE CYS LEU GLU PRO PRO TYR THR GLY PRO CYS LYS ALA ARG ILE ILE ARG 
          TYR PHE TYR ASN ALA LYS ALA GLY LEU CYS GLN THR PHE VAL TYR GLY GLY CYS ARG ALA 
          LYS ARG ASN ASN PHE LYS SER ALA GLU ASP CYS MET ARG THR CYS GLY GLY ALA 
 ***** Message from SEQRDR ***** THE SYSTEM CONTAINS 12 TITRATABLE GROUPS
 THE USER MUST PREDETERMINE THE PROTONATION STATE THROUGH THE SEQUENCE AND RTF
 HIS -  0  HSD -  0  HSE -  0  HSP -  0  ASP -  2  GLU -  2  LYS -  4  TYR -  4
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 THE PATCH 'ACE' WILL BE USED FOR THE FIRST RESIDUE
 THE PATCH 'CT3' WILL BE USED FOR THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.
 GENPSF> Segment   1 has been generated. Its 

In [46]:
# model.ligand+model.phos_ligand+model.co_solvent is the concatenated list of entities
load_ligands(model.ligand+model.phos_ligand+model.co_solvent)
# load_ligands(model.ligand)

[]

## Minimize the Protein Chain First

In [47]:
# Specify nonbonded python object called my_nbonds - this just sets it up
# equivalant CHARMM scripting command: 
# nbonds cutnb 18 ctonnb 13 ctofnb 17 cdie eps 1 atom vatom fswitch vfswitch
non_bonded_script = NonBondedScript(
    cutnb=18.0, ctonnb=13.0, ctofnb=17.0,
    eps=1.0,
    cdie=True,
    atom=True, vatom=True,
    fswitch=True, vfswitch=True
)
# select the C-alpha atoms for harmonic restraints
cons_harm_atoms = pycharmm.SelectAtoms(atom_type='CA')
ener_dict = sd_minimize(300, non_bonded_script, cons_harm_selection=cons_harm_atoms)

  
 CHARMM>     nbonds cutnb 18.0 -
 CHARMM>     ctonnb 13.0 -
 CHARMM>     ctofnb 17.0 -
 CHARMM>     eps 1.0 -
 CHARMM>     cdie -
 CHARMM>     atom -
 CHARMM>     vatom -
 CHARMM>     fswitch -
 CHARMM>     vfswitch

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 18.000 CTEXNB =999.000 CTONNB = 13.000 CTOFNB = 17.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are        0 atom  pairs and        0 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 <MAKINB> with mode   5 found   2556 exclusions and   2364 interactions(1-4)
 <MAKGRP> found    826 group exclusions.
 Generating nonbond list with Exclusion mode = 5
 == PRIMARY == SPACE FOR   405451 ATOM PAIRS AND        0 GROUP PAIRS

 General atom nonbond list generation found:
   285445 ATOM PAIRS WERE FOUND FOR ATOM LIST
     5796 GROUP PAIRS

/home/truman/crimm/crimm/Adaptors/pyCHARMMAdaptors.py:454: UserWarning: Absolute harmonic restraints setup success: True
  warnings.warn(f"Absolute harmonic restraints setup success: {has_cons_harm}")


 UPDECI: Nonbond update at step        50
 Generating nonbond list with Exclusion mode = 5
 == PRIMARY == SPACE FOR   405451 ATOM PAIRS AND        0 GROUP PAIRS

 General atom nonbond list generation found:
   285154 ATOM PAIRS WERE FOUND FOR ATOM LIST
     5772 GROUP PAIRS REQUIRED ATOM SEARCHES

MINI>       50   -811.07500     15.32474      0.90090      0.00087
MINI INTERN>       52.54054    124.86653      8.74555    490.74071      5.89626
MINI CROSS>       -12.17889      0.00000      0.00000      0.00000
MINI EXTERN>     -185.23420  -1299.69285      0.00000      0.00000      0.00000
MINI CONSTR>        3.24135      0.00000      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>       60   -819.97845      8.90344      1.03742      0.00093
MINI INTERN>       53.56299    125.89118      8.72032    488.82762      6.00758
MINI CROSS>       -12.68961      0.00000      0.00000      0.00000
MINI EXTERN>     -189.28994  -130

## Sync Coord with pyCHARMM
We need to update the coords of crimm protein after minimization

In [48]:
## This is the new API for crimm sync coordinates with CHARMM
## The old sync_coord only works in a limited number of situations thus is DEPRECATED
fetch_coords_from_charmm(model.protein+model.ligand+model.phos_ligand+model.co_solvent)

In [49]:
model

NGLWidget()

<OrganizedModel model=4PTI Polypeptide(L)=1 Solvent=1 >
	│
	├───<Polypeptide(L) id=A Residues=58>
	├──────Description: TRYPSIN INHIBITOR
	│
	├───<Solvent id=B Residues=60>
	├──────Residue ID(s): HOH
	├──────Description: water


## Solvation

In [52]:
solvator = Solvator(model)
# we want to keep the crystallograpic water using remove_existing_water=False
# doc string available for Solvator and
added_water = solvator.solvate(
    cutoff=8.0, solvcut=2.1, remove_existing_water=False, orient_coords=False
)
# Solvator.add_balancing_ions will add either sodium (SOD) or chloride (CLA)
# to the solvent according to the total charge of the system
balancing_ion_chain = solvator.add_balancing_ions()

Total charges before adding ions: 6.0
  [Chain A] 6.0


## Doc Strings for Solvator

In [25]:
Solvator?

Init signature: Solvator(entity) -> None
Docstring:     
Solvates a Structure, Model, or Chain level entity with water molecules.
The solvated entity will be returned as a Model level entity. The solvated
entity will be centered in a cubic box with side length equal to the
maximum dimension of the entity plus the cutoff distance. (i.e., Coordinates 
will be oriented using CoordManipulator.orient_coords() before solvation.)
The solvcut distance is the distance from the solute at which water
molecules will be removed. The solvcut distance is used to remove water 
molecules that are too close to the solute. 
If altloc atoms exist in the entity, the first altloc atoms will be used to
determine water molecules location during solvation.

Parameters
----------
entity : Structure, Model, or Chain level entity
    The entity to solvate. If a Structure level entity is provided, the
    first Model will be solvated. If a Model level entity is provided, all 
    chains in the model will be solvat

In [26]:
Solvator.solvate?

Signature:
Solvator.solvate(
    self,
    cutoff=9.0,
    solvcut=2.1,
    remove_existing_water=True,
    orient_coords=True,
    box_type='cube',
) -> crimm.StructEntities.Model.Model
Docstring:
Solvates the entity and returns a Model level entity. The solvated
entity will be centered in a cubic box with side length equal to the
maximum dimension of the entity plus the cutoff distance. (i.e.,
Coordinates will be oriented using CoordManipulator.orient_coords()
before solvation.) The solvcut distance is the distance from the solute
at which water molecules will be removed. The solvcut distance is used
to remove water molecules that are too close to the solute. If altloc
atoms exist in the entity, the first altloc atoms will be used to
determine water molecules location during solvation.

Parameters
----------
cutoff : float, optional
    The distance from the solute to the edge of the cubic box. The
    default is 9.0.
solvcut : float, optional
    The distance from the solute at whic

In [27]:
Solvator.add_balancing_ions?

Signature:
Solvator.add_balancing_ions(
    self,
    present_charge=None,
    cation='SOD',
    anion='CLA',
    skip_undefined=True,
) -> crimm.StructEntities.Chain.Ion
Docstring:
Add balancing ions to the solvated entity to bring total charge to zero.
The default cation is Na+ and the default anion is Cl-. If the entity is
not a solvated entity, a ValueError will be raised. A random selection of
water molecules in the water box will be replaced with balancing ions.
Returns a chain containing the balancing ions.

Parameters
----------
entity : Structure, Model, or Chain level entity
    The solvated entity to add balancing ions to.
present_charge : int, optional
    The present charge of the solvated entity. If None, the charge will be
    calculated from the entity. The default is None. If for any reason you
    want to balance the charge to a non-zero value, you can specify it here.
cation : str, optional
    The cation to use. The default is 'SOD' (Na+).
anion : str, optional
    

## Model after Solvation

Note that we have added 4 chloride ions as chain IA since the total charge was **+4**.<br>
The water box has to be splitted into two chains due to PDB residue number limit to **9999**.

In [53]:
model

NGLWidget()

<OrganizedModel model=4PTI Polypeptide(L)=1 Solvent=2 Ion=1 >
	│
	├───<Polypeptide(L) id=A Residues=58>
	├──────Description: TRYPSIN INHIBITOR
	│
	├───<Solvent id=B Residues=60>
	├──────Residue ID(s): HOH
	├──────Description: water
	│
	├───<Solvent id=WA Residues=3691>
	├──────Description: water
	│
	├───<Ion id=IA Residues=6>
	├──────Description: balancing ions (CLA)


## Load other Entities into CHARMM
Currently Ligand, Nucleophosphate, and CoSolvent can be automatically generated by TopologyLoader. <br>
Thus, if they exist, they can be safely loaded into CHARMM

In [54]:
model.solvent

[<Solvent id=B Residues=60>, <Solvent id=WA Residues=3691>]

In [55]:
load_ions(model.ion)
# This loads both crystallographic water and the water box generated by Solvator
load_water(model.solvent)
# Since crimm cannot build hydrogens on the crystallographic water yet
# we will build them in CHARMM and copy their coords to crimm
create_water_hs_from_charmm(model)

[crimm] Loading ion chain IO00
  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmp0gs0_p1n
 VOPEN> Attempting to open::/tmp/tmp0gs0_p1n::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --     6 RESIDUES
          CLA     CLA     CLA     CLA     CLA     CLA     
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 NO PATCHING WILL BE DONE ON THE FIRST RESIDUE
 NO PATCHING WILL BE DONE ON THE LAST  RESIDUE
 GENPSF> Segment   2 has been generated. Its identifier is IO00.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        2   Number of residues   =       64
         Number of atoms         =      907   Number of groups     =      282
         Number of bonds         =      915   Number of angles     =     1641
         Number of dihedrals     =     2408   Number of impropers  =      156
         Number of cross-terms

In [56]:
# We can visualize the crystallographic water and see the hydrogens are added
model.solvent[0]

NGLWidget()

<Solvent id=B Residues=60>
  Residue ID(s): HOH
  Description: water


## Set up PBC and Minimize Water

In [57]:
# organize segids and ion types for image and cons_fix
non_solvent_segids = set()
all_ion_types = set()
for chain in model:
    if chain.chain_type == 'Solvent':
        continue
    elif chain.chain_type == 'Ion':
        for res in chain:
            all_ion_types.add(res.resname)
    else:
        for res in chain:
            non_solvent_segids.add(res.segid)

In [58]:
# anything but solvent or ions in the model
non_solvent_segids

{'PROA'}

In [59]:
# all types of ions loaded in pyCHARMM by crimm
all_ion_types

{'CLA'}

In [60]:
# CHARMM scripting: crystal define cubic @boxsize @boxsize @boxsize 90 90 90
crystal.define_cubic(solvator.box_dim)
# CHARMM scripting: crystal build cutoff @boxhalf noper 0
crystal.build(solvator.box_dim/2)

 Crystal Parameters : Crystal Type = CUBI
           A     =   50.66930 B    =   50.66930 C     =   50.66930
           Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
 XBUILD> Building all transformations with a minimum atom-atom
         contact distance of less than   25.33 Angstroms.

 Range of Grid Search for Transformation     1 :
 Lattice Vector A    -2 TO     2
 Lattice Vector B    -2 TO     2
 Lattice Vector C    -2 TO     2


 The number of transformations generated =    26


 Number  Symop   A   B   C   Distance

      1      1  -1  -1  -1     5.6520
      2      1  -1   0  -1     5.4010
      3      1  -1   1  -1     7.1318
      4      1   0  -1  -1     2.5438
      5      1   0   0  -1     1.1614
      6      1   0   1  -1     3.8318
      7      1  -1  -1   0     3.4304
      8      1  -1   0   0     1.9491
      9      1  -1   1   0     3.9638
     10      1   0  -1   0     1.3553
     11      1   0   1   0     1.3553
     12      1  -1  -1   1     4.2161
     1

1

In [61]:
# Turn on image centering - bysegment for protein, by residue for solvent and ions
# CHARMM scripting: image byseg xcen 0 ycen 0 zcen 0 select segid SEGID end
for segid in non_solvent_segids:
    image.setup_segment(0.0, 0.0, 0.0, segid)
# CHARMM scripting: image byres xcen 0 ycen 0 zcen 0 select resname tip3 end
image.setup_residue(0.0, 0.0, 0.0, 'TIP3')
# CHARMM scripting: image byres xcen 0 ycen 0 zcen 0 select resname ion_type end
for ion_type in all_ion_types:
    image.setup_residue(0.0, 0.0, 0.0, ion_type)

 select>    901 atoms have been selected out of   12160
 IMAGE CENTERING ON FOR SOME ATOMS
 select>  11253 atoms have been selected out of   12160
 IMAGE CENTERING ON FOR SOME ATOMS
 select>      6 atoms have been selected out of   12160
 IMAGE CENTERING ON FOR SOME ATOMS


In [62]:
# Now specify nonbonded cutoffs for solvated box
cutnb = min(solvator.box_dim/2, 12)
cutim = cutnb
ctofnb = cutnb - 1.0
ctonnb = cutnb - 3.0

# Another nbonds example
# CHARMM scripting: nbonds cutnb @cutnb cutim @cutim ctofnb @ctofnb ctonnb @ctonnb -
#        inbfrq -1 imgfrq -1
non_bonded_script = pycharmm.NonBondedScript(
    cutnb=cutnb, cutim=cutim, ctonnb=ctonnb, ctofnb=ctofnb,
    eps=1.0,
    cdie=True,
    atom=True, vatom=True,
    fswitch=True, vfswitch=True,
    inbfrq=-1, imgfrq=-1
)

In [63]:
# We want to fix the protein and ligands and minimize the solvent to "fit"
# Select everything but solvent and ions
cons_fix_atoms = pycharmm.SelectAtoms()
for segid in non_solvent_segids:
    cons_fix_atoms |= pycharmm.SelectAtoms(seg_id=segid)

# Minimize the solvent positions with periodic boundary conditions using steepest descents
ener_dict = sd_minimize(200, non_bonded_script, cons_fix_selection=cons_fix_atoms)

  
 CHARMM>     nbonds cutnb 12 -
 CHARMM>     cutim 12 -
 CHARMM>     ctonnb 9.0 -
 CHARMM>     ctofnb 11.0 -
 CHARMM>     eps 1.0 -
 CHARMM>     cdie -
 CHARMM>     atom -
 CHARMM>     vatom -
 CHARMM>     fswitch -
 CHARMM>     vfswitch -
 CHARMM>     inbfrq -1 -
 CHARMM>     imgfrq -1

 SELECTED IMAGES ATOMS BEING CENTERED ABOUT  0.000000  0.000000  0.000000

 <MKIMAT2>: updating the image atom lists and remapping
 Transformation   Atoms  Groups  Residues  Min-Distance
    1  N1N1N1R1 has      51      17      17        2.51
    2  N1Z0N1R1 has     447     149     149        0.25
    3  N1P1N1R1 has      51      17      17        3.76
    4  Z0N1N1R1 has     483     161     161        1.43
    5  Z0Z0N1R1 has    2819     941     941        0.08
    6  Z0P1N1R1 has     449     151     151        0.47
    7  N1N1Z0R1 has     522     174     174        0.27
    8  N1Z0Z0R1 has    2804     930     918        0.14
    9  N1P1Z0R1 has     444     148     148        1.23
   10  Z0N1Z0R1 ha

/home/truman/crimm/crimm/Adaptors/pyCHARMMAdaptors.py:462: UserWarning: Atom fix constraint setup success: True
  warnings.warn(f"Atom fix constraint setup success: {has_cons_fix}")


 HBFIND-exclusions:******* due to distance cutoff,       0 due to angle cutoff
                         0 primary donor to image acceptor hbonds found
 HBFIND-exclusions:******* due to distance cutoff,       0 due to angle cutoff
                         0 image donor to primary acceptor hbonds found
                         0 unique image hbonds found
 HBEDIT-deletions:       0 due to duplications,          0 due to best-option,
                         0 due to fixed atoms,           0 due to exclusions
 HBEDIT: currently     0 hydrogen bonds present


 STEEPD> An energy minimization has been requested.

 NSTEP  =          200   NPRINT =           10
 STEP   =    0.0200000   TOLFUN =    0.0010000
 TOLGRD =    0.0010000   TOLSTP =    0.0000000

MINI MIN: Cycle      ENERgy      Delta-E         GRMS    Step-size
MINI INTERN:          BONDs       ANGLes       UREY-b    DIHEdrals    IMPRopers
MINI EXTERN:        VDWaals         ELEC       HBONds          ASP         USER
MINI IMAGES:     

In [64]:
fetch_coords_from_charmm(model)
model

NGLWidget()

<OrganizedModel model=4PTI Polypeptide(L)=1 Solvent=2 Ion=1 >
	│
	├───<Polypeptide(L) id=A Residues=58>
	├──────Description: TRYPSIN INHIBITOR
	│
	├───<Solvent id=B Residues=60>
	├──────Residue ID(s): HOH
	├──────Description: water
	│
	├───<Solvent id=WA Residues=3691>
	├──────Description: water
	│
	├───<Ion id=IA Residues=6>
	├──────Description: balancing ions (CLA)


In [65]:
pcm_write.coor_card(f'{PDBID}.crd')
pcm_write.psf_card(f'{PDBID}.psf')

  
 CHARMM>     write name 4E0J.crd -
 CHARMM>     coor card
 VOPEN> Attempting to open::4e0j.crd::
 RDTITL>  
 RDTITL> No title read.
 VCLOSE: Closing unit   91 with status "KEEP"
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     write name 4E0J.psf -
 CHARMM>     psf card
 VOPEN> Attempting to open::4e0j.psf::
 RDTITL>  
 RDTITL> No title read.
 VCLOSE: Closing unit   91 with status "KEEP"
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
